# MySQL-server has gone away解决方法

mysql有一个限制: mysql默认会断开超过8小时未活跃过的连接.如果连续8小时无连接(没有任何数据库操作),mysql会自动断开连接,这时你再访问数据库则会报出"MySQL server has gone away"错误. 如果mysql重启或断开过连接，那么依然会在第一次时报"MySQL server has gone away"，

如何解决:
1. 设置`pool_recycle`参数, 

该参数会在每个session调用执行sql前校验当前时间与上一次连接时间间隔是否超过pool_recycle，如果超过就会重置。
`engine = create_engine('mysql+mysqldb://...', pool_recycle=3600)`

2. 设置心跳

如需要非常严格的mysql断线重连策略，可以设置心跳:

```
from sqlalchemy import create_engine, event
from sqlalchemy.exc import DisconnectionError

def checkout_listener(dbapi_con, con_record, con_proxy):
    try:
        try:
            dbapi_con.ping(False)
        except TypeError:
            dbapi_con.ping()
    except dbapi_con.OperationalError as exc:
        if exc.args[0] in (2006, 2013, 2014, 2045, 2055):
            raise DisconnectionError()
        else:
            raise

db_engine = create_engine(DATABASE_CONNECTION_INFO, pool_size=100, pool_recycle=3600)
event.listen(db_engine, 'checkout', checkout_listener)

```

[心跳设置参考](https://stackoverflow.com/questions/18054224/python-sqlalchemy-mysql-server-has-gone-away)